In [1]:
from functools import partial

from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    when, regexp_replace, flatten, explode,
    struct, create_map, array, col
)
from pyspark.sql.types import (
    StructType, StructField,
    StringType, IntegerType, ArrayType, TimestampType
)

In [2]:
spark = SparkSession.builder.appName("chap2").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/21 23:42:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Cleaning & Preparing Your Data

In [3]:
df1 = spark.createDataFrame(
    [
        ("Alisson", "GK", 1),
        ("Walter", "DM", 3),
        ("Virgil", "DF", 4),
        ("Ibrahim", "DF", None),
        ("Endo", None, 3),
        ("Alisson", "GK", 1),
        ("Luis", "LW", 0)
    ], schema=["name", "position", "id"]
)
df1

DataFrame[name: string, position: string, id: bigint]

In [4]:
df1.show()

+-------+--------+----+
|   name|position|  id|
+-------+--------+----+
|Alisson|      GK|   1|
| Walter|      DM|   3|
| Virgil|      DF|   4|
|Ibrahim|      DF|null|
|   Endo|    null|   3|
|Alisson|      GK|   1|
|   Luis|      LW|   0|
+-------+--------+----+



## Aliasing & Renaming Columns

In [5]:
df1.select(df1["name"].alias("player")).show()

+-------+
| player|
+-------+
|Alisson|
| Walter|
| Virgil|
|Ibrahim|
|   Endo|
|Alisson|
|   Luis|
+-------+



In [6]:
df1.withColumnRenamed("id", "number").show()

+-------+--------+------+
|   name|position|number|
+-------+--------+------+
|Alisson|      GK|     1|
| Walter|      DM|     3|
| Virgil|      DF|     4|
|Ibrahim|      DF|  null|
|   Endo|    null|     3|
|Alisson|      GK|     1|
|   Luis|      LW|     0|
+-------+--------+------+



## Duplicate Values

Show only unique rows:

In [7]:
df1.distinct().show()

+-------+--------+----+
|   name|position|  id|
+-------+--------+----+
|Alisson|      GK|   1|
| Walter|      DM|   3|
| Virgil|      DF|   4|
|Ibrahim|      DF|null|
|   Endo|    null|   3|
|   Luis|      LW|   0|
+-------+--------+----+



Filter out duplicates based on entire rows (same as using `distinct()`):

In [8]:
df1.dropDuplicates().show()

+-------+--------+----+
|   name|position|  id|
+-------+--------+----+
|Alisson|      GK|   1|
| Walter|      DM|   3|
| Virgil|      DF|   4|
|Ibrahim|      DF|null|
|   Endo|    null|   3|
|   Luis|      LW|   0|
+-------+--------+----+



Filter out duplicates based on a subset of columns:

In [9]:
df1.dropDuplicates(["id"]).show()

+-------+--------+----+
|   name|position|  id|
+-------+--------+----+
|Ibrahim|      DF|null|
|   Luis|      LW|   0|
|Alisson|      GK|   1|
| Walter|      DM|   3|
| Virgil|      DF|   4|
+-------+--------+----+



## Working With Nulls

Filter rows based on the existence or absence of nulls in a specified column:

In [10]:
df1.filter(df1["id"].isNull()).show()

+-------+--------+----+
|   name|position|  id|
+-------+--------+----+
|Ibrahim|      DF|null|
+-------+--------+----+



In [11]:
df1.filter(
    df1["id"].isNotNull() & df1["position"].isNotNull()
).show()

+-------+--------+---+
|   name|position| id|
+-------+--------+---+
|Alisson|      GK|  1|
| Walter|      DM|  3|
| Virgil|      DF|  4|
|Alisson|      GK|  1|
|   Luis|      LW|  0|
+-------+--------+---+



Create a null label column:

In [12]:
df1.select("*", df1["id"].isNull().alias("no_id")).show()

+-------+--------+----+-----+
|   name|position|  id|no_id|
+-------+--------+----+-----+
|Alisson|      GK|   1|false|
| Walter|      DM|   3|false|
| Virgil|      DF|   4|false|
|Ibrahim|      DF|null| true|
|   Endo|    null|   3|false|
|Alisson|      GK|   1|false|
|   Luis|      LW|   0|false|
+-------+--------+----+-----+



Account for cases when nulls are set to alternative values:

In [13]:
df1.withColumn(
    "id",
    when(df1["id"] == 0, None).otherwise(df1["id"])
).show()

+-------+--------+----+
|   name|position|  id|
+-------+--------+----+
|Alisson|      GK|   1|
| Walter|      DM|   3|
| Virgil|      DF|   4|
|Ibrahim|      DF|null|
|   Endo|    null|   3|
|Alisson|      GK|   1|
|   Luis|      LW|null|
+-------+--------+----+



## Using RegEx

In [14]:
df2 = spark.createDataFrame(
    [
        ("Alisson", "Math"),
        ("Virgil", "Computer Science"),
        ("Ibrahim", "English"),
        ("Alexis", "Computer Science")
    ], schema=["name", "class"]
)
df2.show()

+-------+----------------+
|   name|           class|
+-------+----------------+
|Alisson|            Math|
| Virgil|Computer Science|
|Ibrahim|         English|
| Alexis|Computer Science|
+-------+----------------+



Replace one value with an alternative:

In [15]:
df2.select(
    df2["class"].alias("initial_choice"),
    regexp_replace("class", "English", "Design").alias("final_choice")
).show()

+----------------+----------------+
|  initial_choice|    final_choice|
+----------------+----------------+
|            Math|            Math|
|Computer Science|Computer Science|
|         English|          Design|
|Computer Science|Computer Science|
+----------------+----------------+



Replace all values in a column:

In [16]:
df2.withColumn(
    "class",
    regexp_replace("class", "^Computer", "Data")
).show()

+-------+------------+
|   name|       class|
+-------+------------+
|Alisson|        Math|
| Virgil|Data Science|
|Ibrahim|     English|
| Alexis|Data Science|
+-------+------------+



## Type-Casting

In [17]:
df3 = spark.createDataFrame(
    [
        ("Alisson", "1"),
        ("Virgil", "4"),
        ("Ibrahim", "5"),
        ("Alexis", "10")
    ], schema=["name", "id"]
)
df3.printSchema()

root
 |-- name: string (nullable = true)
 |-- id: string (nullable = true)



In [18]:
df3.select("name", df3["id"].cast("int").alias("int")).printSchema()

root
 |-- name: string (nullable = true)
 |-- int: integer (nullable = true)



## Complex Data Types & Related Functions

Struct type, consisting of a list of `StructField`s, is the data type representing a `Row` in Spark:

In [19]:
schema = StructType([
    StructField(
        "friend",
        StructType([
            StructField("name", StringType(), False),
            StructField("age", IntegerType())
        ])
    ),
    StructField("skills", ArrayType(ArrayType(StringType()))),
    StructField("id", IntegerType(), False)
])
data = [
    (("Alex", 20), [["data", "research"], ["Python", "SQL"]], 1),
    (("Eugene", 27), [["business", "research"], ["Excel", None]], 2)
]
df4 = spark.createDataFrame(data, schema)
df4.show(truncate=False)
df4.printSchema()

+------------+-------------------------------------+---+
|friend      |skills                               |id |
+------------+-------------------------------------+---+
|{Alex, 20}  |[[data, research], [Python, SQL]]    |1  |
|{Eugene, 27}|[[business, research], [Excel, null]]|2  |
+------------+-------------------------------------+---+

root
 |-- friend: struct (nullable = true)
 |    |-- name: string (nullable = false)
 |    |-- age: integer (nullable = true)
 |-- skills: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: string (containsNull = true)
 |-- id: integer (nullable = false)



Create a struct column from existing columns:

In [20]:
df5 = df4.select(struct("id", "friend").alias("friend"))
df5.show()

+-----------------+
|           friend|
+-----------------+
|  {1, {Alex, 20}}|
|{2, {Eugene, 27}}|
+-----------------+



In [21]:
df5.select("friend.friend.name").show()

+------+
|  name|
+------+
|  Alex|
|Eugene|
+------+



Access values in an array using bracket notation:

In [22]:
df4.select(df4["skills"][0][0].alias("skill")).show()

+--------+
|   skill|
+--------+
|    data|
|business|
+--------+



Flatten a nested array:

In [23]:
df4.select(
    flatten(df4["skills"]).alias("skills")
).show(truncate=False)

+---------------------------------+
|skills                           |
+---------------------------------+
|[data, research, Python, SQL]    |
|[business, research, Excel, null]|
+---------------------------------+



Extract an array into rows:

In [24]:
df4.select(
    "friend.name", explode("skills").alias("skill")
).show()

+------+--------------------+
|  name|               skill|
+------+--------------------+
|  Alex|    [data, research]|
|  Alex|       [Python, SQL]|
|Eugene|[business, research]|
|Eugene|       [Excel, null]|
+------+--------------------+



Another example of using `pyspark.sql.functions.explode()`, this time on a hash structure:

In [25]:
df6 = spark.createDataFrame(
    [(1, {"Alex": "data"}), (2, {"Eugene": "business"})],
    schema=["name", "skill"]
)
df6.show()
df6.printSchema()

+----+--------------------+
|name|               skill|
+----+--------------------+
|   1|      {Alex -> data}|
|   2|{Eugene -> business}|
+----+--------------------+

root
 |-- name: long (nullable = true)
 |-- skill: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)



In [26]:
df6.select("name", explode("skill")).show()

+----+------+--------+
|name|   key|   value|
+----+------+--------+
|   1|  Alex|    data|
|   2|Eugene|business|
+----+------+--------+



Create map-type and array-type columns from existing columns:

In [27]:
df7 = df4.select(
    create_map("id", "friend.name").alias("friend_map"),
    array("friend.name", "friend.age").alias("friend_array")
)
df7.show()
df7.printSchema()

+-------------+------------+
|   friend_map|friend_array|
+-------------+------------+
|  {1 -> Alex}|  [Alex, 20]|
|{2 -> Eugene}|[Eugene, 27]|
+-------------+------------+

root
 |-- friend_map: map (nullable = false)
 |    |-- key: integer
 |    |-- value: string (valueContainsNull = true)
 |-- friend_array: array (nullable = false)
 |    |-- element: string (containsNull = true)



# Practical Lab

In [28]:
bronze_sales = spark.createDataFrame(
    [
        ("1", "LA", "2000-01-01", 5, 1400),
        ("2", "LA", "1998-2-01", 4, 1500),
        ("2", "LA", "1998-2-01", 4, 1500),
        ("3", "LA", "1997-4-01", 6, 1300),
        ("4", "LA", "2005-5-01", 2, 1100),
        ("NA", "LA", "2013-6-01", 1, 1200)
    ], schema=["sales_id", "city", "dat", "clerk_id", "total_sales"]
)
bronze_sales.show()

+--------+----+----------+--------+-----------+
|sales_id|city|       dat|clerk_id|total_sales|
+--------+----+----------+--------+-----------+
|       1|  LA|2000-01-01|       5|       1400|
|       2|  LA| 1998-2-01|       4|       1500|
|       2|  LA| 1998-2-01|       4|       1500|
|       3|  LA| 1997-4-01|       6|       1300|
|       4|  LA| 2005-5-01|       2|       1100|
|      NA|  LA| 2013-6-01|       1|       1200|
+--------+----+----------+--------+-----------+



Fix column name:

In [29]:
bronze_sales = bronze_sales.withColumnRenamed("dat", "date")
bronze_sales.columns

['sales_id', 'city', 'date', 'clerk_id', 'total_sales']

Correctly identify nulls and drop them (note that the `sales_id` column has null values as `NA` strings):

In [30]:
bronze_sales = (
    bronze_sales.replace("NA", None, "sales_id")
        .filter(col("sales_id").isNotNull())
)
bronze_sales.show()

+--------+----+----------+--------+-----------+
|sales_id|city|      date|clerk_id|total_sales|
+--------+----+----------+--------+-----------+
|       1|  LA|2000-01-01|       5|       1400|
|       2|  LA| 1998-2-01|       4|       1500|
|       2|  LA| 1998-2-01|       4|       1500|
|       3|  LA| 1997-4-01|       6|       1300|
|       4|  LA| 2005-5-01|       2|       1100|
+--------+----+----------+--------+-----------+



In [31]:
spark.stop()